In [1]:
import os
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

import dataset
from dataset import CircularLearningRate

from tensorflow import config
assert config.list_physical_devices('GPU')

In [99]:
BATCH_SIZE = 32
MAXLEN = 64

LETTERS_SIZE = len(dataset.letters_table)
NIQQUD_SIZE = len(dataset.niqqud_table)
DAGESH_SIZE = len(dataset.dagesh_table)
SIN_SIZE = len(dataset.sin_table)
KINDS_SIZE = len(dataset.KINDS)

def build_model(EMBED_DIM=45, UNITS=190):

    layer = input_text = keras.Input(batch_shape=(None, MAXLEN), batch_size=BATCH_SIZE)
    
    layer = layers.Embedding(LETTERS_SIZE, EMBED_DIM, input_length=MAXLEN, mask_zero=True)(layer)
    layer = layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True, dropout=0.0), merge_mode='sum')(layer)
    layer = layers.add([layer, 
            layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True, dropout=0.0), merge_mode='sum')(layer)])
    
    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
    ]
    model = keras.Model(inputs=[input_text], outputs=outputs)

    jsmodel = model 
    # keras.utils.plot_model(model, to_file='model.png')
    return model, jsmodel

model, jsmodel = build_model()

model.summary()
model.save_weights('./checkpoints/uninit')

Model: "model_18"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           [(None, 64)]         0                                            
__________________________________________________________________________________________________
embedding_18 (Embedding)        (None, 64, 45)       1980        input_19[0][0]                   
__________________________________________________________________________________________________
bidirectional_36 (Bidirectional (None, 64, 190)      358720      embedding_18[0][0]               
__________________________________________________________________________________________________
bidirectional_37 (Bidirectional (None, 64, 190)      579120      bidirectional_36[0][0]           
___________________________________________________________________________________________

In [3]:
def fit(train_validation, scheduler, verbose=1):
    train, valid = train_validation
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    callbacks = []
    if isinstance(scheduler, CircularLearningRate):
        scheduler.set_dataset(train, BATCH_SIZE)
    if scheduler:
        callbacks.append(scheduler)
        
    x  = train.normalized
    vx = valid.normalized
    y  = {'N': train.niqqud, 'D': train.dagesh, 'S': train.sin, 'C': train.normalized }
    vy = {'N': valid.niqqud, 'D': valid.dagesh, 'S': valid.sin, 'C': valid.normalized }
    return model.fit(x, y, validation_data=(vx, vy), batch_size=BATCH_SIZE, epochs=1, verbose=verbose, callbacks=callbacks)


In [4]:
def load_data(source, maxlen=MAXLEN, validation=0.1):
    filenames = [os.path.join('texts', f) for f in source]
    train, valid = dataset.load_data(filenames, validation, maxlen=maxlen)
    return train, valid

In [29]:
data_other = load_data(['biblical', 'garbage', 'poetry'])

In [30]:
data_rabanit = load_data(['rabanit'])

In [31]:
data_pre_modern = load_data(['pre_modern'])

In [105]:
data_modern = load_data(validation=0.1, source=['modern'])

In [100]:
model.load_weights('./checkpoints/uninit')
history = fit(data_other, scheduler=CircularLearningRate(20e-4, 50e-4, 5e-4))  # val_N_accuracy: 0.9063 - val_D_accuracy: 0.9712
model.save_weights('./checkpoints/other')

Train on 49310 samples, validate on 934 samples
49310/49310 [==============================] - 60s 1ms/sample - loss: 0.5874 - N_loss: 0.4669 - D_loss: 0.1011 - S_loss: 0.0195 - N_accuracy: 0.8378 - D_accuracy: 0.9602 - S_accuracy: 0.9946 - val_loss: 0.3824 - val_N_loss: 0.3025 - val_D_loss: 0.0732 - val_S_loss: 0.0059 - val_N_accuracy: 0.8949 - val_D_accuracy: 0.9711 - val_S_accuracy: 0.99830411 - N_loss: 1.5687 - D_loss: 0.3041 - S_loss: 0.1683 - N_accuracy: 0.4953 - D_accuracy: 0.9027 - S_accuracy: 0.960 - ETA: 2:35 - loss: 2.0252 - N_loss: 1.5573 - D_loss: 0.3024 - S_loss: 0.1655 - N_accuracy: 0.4978 - D_accu - ETA: 1:46 - loss: 1.6352 - N_loss: 1.2743 - D_loss: 0.2546 - S_loss: 0.1063 - N_accuracy: 0.5732 - D_accuracy: 0.9099 -  - ETA: 1:32 - loss: 1.5051 - N_loss: 1.1777 - D_loss: 0.2373 - S_loss: 0.0902 - N_accur - ETA: 25s - loss: 0.7692 - N_loss: 0.6119 - D_loss: 0.1287 - - ETA: 22s - loss: 0.7381 - N_loss: 0.5870 - D_loss: 0.1241 - S_loss: 0.0269 - N_accu - ETA: 19s - loss: 0

In [101]:
model.load_weights('./checkpoints/other')
history = fit(data_rabanit, scheduler=CircularLearningRate(20e-4, 50e-4, 5e-4))  # val_N_accuracy: 0.9348 - val_D_accuracy: 0.9806
model.save_weights('./checkpoints/rabanit')

Train on 81160 samples, validate on 841 samples
81160/81160 [==============================] - 93s 1ms/sample - loss: 0.1625 - N_loss: 0.1246 - D_loss: 0.0347 - S_loss: 0.0032 - N_accuracy: 0.9571 - D_accuracy: 0.9872 - S_accuracy: 0.9993 - val_loss: 0.2572 - val_N_loss: 0.1960 - val_D_loss: 0.0580 - val_S_loss: 0.0031 - val_N_accuracy: 0.9367 - val_D_accuracy: 0.9805 - val_S_accuracy: 0.9991: 1:13 - loss: 0.2122 - N_loss: 0.1648 - D_loss: 0.0437 - S_loss: 0.0037 - N_accuracy: 0.9424 - D_accuracy: 0.9836 - S_accuracy: 0. - ETA: 1:12 - loss: 0.2118 - N_loss: 0.1645 - D_loss: 0.0436 - S_ - ETA: 1:04 - loss: 0.2062 - N_loss: 0.1598 - D_loss: 0.0427  - ETA: 58s - loss: 0.2022 - N_loss: 0.1566 - D_loss: 0.0420 - S_loss: 0.0037 - N_accuracy: 0.9455 - D_accuracy: - ETA: 56s - loss: 0.2015 - N_loss: 0.1560 - D_loss: 0.0419 - S_loss: 0.0037 - N_accuracy: 0.9458 - D_accuracy: 0.9 - ETA: 55s - loss: 0.2007 - N_loss: 0.1553 - D_loss: 0.0417 - S_loss: 0.0037  - ETA: 52s - loss: 0.1986 - N_loss: 0.1

In [102]:
model.load_weights('./checkpoints/rabanit')
history = fit(data_pre_modern, scheduler=CircularLearningRate(20e-4, 40e-4, 0.1e-4))  # val_N_accuracy: 0.9407 - val_D_accuracy: 0.9810
model.save_weights('./checkpoints/pre_modern')

Train on 66276 samples, validate on 5761 samples
66276/66276 [==============================] - 80s 1ms/sample - loss: 0.2612 - N_loss: 0.1996 - D_loss: 0.0554 - S_loss: 0.0062 - N_accuracy: 0.9322 - D_accuracy: 0.9777 - S_accuracy: 0.9983 - val_loss: 0.2244 - val_N_loss: 0.1739 - val_D_loss: 0.0455 - val_S_loss: 0.0051 - val_N_accuracy: 0.9454 - val_D_accuracy: 0.9822 - val_S_accuracy: 0.998510 - D_loss: 0.0797 - S_loss: 0.0099 - N_accuracy: 0.8977 - D_accuracy: 0 - ETA: 3:11 - loss: 0.3852 - N_loss: 0.3008 - D_loss: 0.0754 - S_loss: 0.0091 - N_accuracy: 0.9015 - D_accuracy: 0.9705 - S_accuracy: 0.9 - ETA: 3:04 - loss: 0.3824 - N_loss: 0.2982 - D_loss: 0.0750 -  - ETA: 1:46 - loss: 0.3450 - N_loss: 0.2685 - D_loss: 0.0687 - S_loss: 0.0079 - N_accuracy: 0.9101 - D_accuracy: 0.9726 - S_accuracy:  - ETA: 1:44 - loss: 0.3429 - N_loss: 0.2669 - D_loss: 0.0682 - S_loss: 0.0078 - N_accuracy: 0.9106 - D_accuracy: 0.9728 - S_a - ETA: 1:37 - loss: 0.3387 - N_loss: 0.2634 - D_loss: 0.0675 - S_lo

In [106]:
model.load_weights('./checkpoints/pre_modern')
history = fit(data_modern, scheduler=CircularLearningRate(2e-3, 3e-3, 0.5e-3))  # val_N_accuracy: 0.9591 - val_D_accuracy: 0.9856
# history = fit(data_modern, scheduler=CircularLearningRate(1e-4, 7e-4, 1e-4))  # val_N_accuracy: 0.9605 - val_D_accuracy: 0.9863
model.save_weights('./checkpoints/modern')

Train on 15948 samples, validate on 1728 samples
15948/15948 [==============================] - 25s 2ms/sample - loss: 0.1809 - N_loss: 0.1340 - D_loss: 0.0437 - S_loss: 0.0031 - N_accuracy: 0.9550 - D_accuracy: 0.9837 - S_accuracy: 0.9993 - val_loss: 0.2013 - val_N_loss: 0.1408 - val_D_loss: 0.0583 - val_S_loss: 0.0022 - val_N_accuracy: 0.9515 - val_D_accuracy: 0.9779 - val_S_accuracy: 0.9995uracy: 0.9821 - S_ - ETA: 6s - loss: 0.1947 - N_loss: 0.1446 - D_loss: 0.0468 - S_loss: 0.0033 - N_accuracy - ETA: 1s - loss: 0.1833 - N_loss: 0.1359 - D_loss: 0.0443 - S_loss: 0.0031 - N_accuracy: 0.9544 - D_accuracy: 0.9835 - S_ac


In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=2)

for n, v in enumerate(['accuracy', 'loss'], 0):
    for n1, t in enumerate(['D', 'N'], 0):
        p = ax[n][n1]
        p.plot(history.history[t + '_' + v][0:])
        p.plot(history.history['val_' + t + '_' +  v][0:])
        p.legend([t + '_Train', t + '_Test'], loc='center right')

plt.tight_layout()

In [ ]:
jsmodel.summary()

In [108]:
import tensorflowjs as tfjs
jsmodel.load_weights('./checkpoints/modern')
tfjs.converters.save_keras_model(jsmodel, '.')

In [107]:
model.load_weights('./checkpoints/modern')

def print_predictions(data, k):
    s = slice(k*BATCH_SIZE, (k+1)*BATCH_SIZE)
    batch = data.normalized[s]
    prediction = model.predict(batch)
    [actual_niqqud, actual_dagesh, actual_sin] = [dataset.from_categorical(prediction[0]), dataset.from_categorical(prediction[1]), dataset.from_categorical(prediction[2])]
    [expected_niqqud, expected_dagesh, expected_sin] = [data.niqqud[s], data.dagesh[s], data.sin[s]]
    actual = dataset.merge(batch, ns=actual_niqqud, ds=actual_dagesh, ss=actual_sin)
    expected = dataset.merge(batch, ns=expected_niqqud, ds=expected_dagesh, ss=expected_sin)
    for i, (a, e) in enumerate(zip(actual, expected)):
        print('מצוי: ', a)
        print('רצוי: ', e)
        print()

print_predictions(data_modern[1], 10)

מצוי:  רַדְיוֹאַקְטִיבִיִּים וְלָכֵן פָּלַט קְרִינָה חֲזָקָה, לְפִי הַמְּשׁוּעָר פִּי אֶלֶף מְעוּצְמֶת 
רצוי:  רַדְיוֹאַקְטִיבִיִּים וְלָכֵן פָּלַט קְרִינָה חֲזָקָה, לְפִי הַמְּשׁוֹעָר פִּי אֶלֶף מֵעוֹצְמַת 

מצוי:  מַאֲזִין לְאַחַת מֵהַיְּצִירוֹת הַמְּטַלְטָלוֹת בְּיוֹתֵר שֶׁנֶּחְשַׁפְתִּי אֲלֵיהֶן אִי פַּעַם. אֲנִי 
רצוי:  מַאֲזִין לְאַחַת מֵהַיְּצִירוֹת הַמְּטַלְטְלוֹת בְּיוֹתֵר שֶׁנֶּחְשַׂפְתִּי אֲלֵיהֶן אֵי פַּעַם. אֲנִי 

מצוי:  פָּתְחָה אֶת הָאָרוֹן וּמָצְאָה בְּלוּן אָדָם בְּלוּן צָהַב וּבָלוּן יָרַק אָמְרָה נוֹעָה: 
רצוי:  פָּתְחָה אֶת הָאָרוֹן וּמָצְאָה בָּלוֹן אָדָם בָּלוֹן צָהֹב וּבָלוֹן יָרַק אָמְרָה נוֹעָה: 

מצוי:  בִּזְמַן שֶׁבְּרַחֲבֵי הָאָרֶץ שׁוֹמְעִים בְּגִלְגְּלַ"צ אֶת הָרִינְגְטוֹן "הוּא תָּקוּעַ בְּפֶקָק", 
רצוי:  בִּזְמַן שֶׁבְּרַחֲבֵי הָאָרֶץ שׁוֹמְעִים בְּגַּלְגַּלָּ"צ אֶת הָרִינְגְּטוֹן "הוּא תָּקוּעַ בִּפְקָק", 

מצוי:  שָׁלוֹם"), נִמְרוֹד רוֹבִּי, רַפִּי בַּרְזִילִי, רוֹנֵן חֲכִימִי, תְּמֵי אֱיָיל, אוּדִי נָטַע 
רצוי:  שָׁלוֹם"), נִמְרוֹד רוּבִּי, רָפִי בַּ

In [ ]:
shutil.rmtree(os.sep.join([tempfile.gettempdir(), '.tensorboard-info']), ignore_errors=True)
shutil.rmtree('logs', ignore_errors=True)
os.makedirs('logs')
# %tensorboard --logdir logs

In [ ]:
print(data_modern[1].text[0])
print(data_modern[1].text[1])